In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import gym
from gym.envs.registration import register

In [4]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback

In [5]:
from assembly_learning.utils import GraphFeatureExtractor

In [9]:
furniture = "ivar"
reward = "partial_th"
max_ep_length = 10
points_threshold = 0.015
lr = 6e-4
total_timesteps = 2e5
verbose = 1

In [10]:
reward_type_list = ["partial_th", "partial_chamfer", "partial_chamfer_change", 
                    "partial_end_chamfer", "partial_th_end", "partial_th_corr", 
                    "partial_th_comp"]

furniture_dict = {"agne": (1, "2_part2", 10, 5), 
                  "bernhard": (2, "2_part2", 10, 5), 
                  "bertil": (3, "0_part0", 1, 5), 
                  "ivar": (4, "0_part0", 10, 5), "mikael": (5, "3_part3", 155, 5), "sivar": (6, "0_part0", 1, 5), 
                  "liden": (7, "9_part9", 10, 5), "swivel": (8, "3_chair_seat", 10, 5), "klubbo": (9, "5_part5", 30, 10), 
                  "lack": (10, "4_part4", 10, 10), "tvunit": (11, "4_part4", 150, 10)}




In [11]:
env_name = 'AssembleRL-{}-{}-v0'.format(furniture, reward)

In [12]:
register(
    id= env_name,
    entry_point='assembly_learning.envs:AssemblyCloudFeatures',
    kwargs={
        "rew_type": reward,
        "points_threshold": points_threshold,
        "num_threshold": furniture_dict[furniture][2],
        "main_object": furniture_dict[furniture][1],
        "pc_sample_size": furniture_dict[furniture][3]*1000,
        "max_ep_length": max_ep_length,
        "furniture_id": furniture_dict[furniture][0],
    },          
)

In [15]:
eval_env = gym.make(env_name)
eval_callback = EvalCallback(eval_env, 
                             best_model_save_path="./logs/{}-{}/".format(args.furniture, args.reward),
                             log_path="./logs/{}-{}/".format(args.furniture, args.reward), 
                             eval_freq=1000,
                             deterministic=True, 
                             render=False)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
policy_kwargs = dict(activation_fn=torch.nn.ReLU,
                net_arch=[dict(pi=[256, 256], vf=[256, 256])],
                features_extractor_class=GraphFeatureExtractor,
                features_extractor_kwargs=dict(features_dim=7*eval_env.n_objects*256, pooling=False),)